# Azure Privileged Identity Management

[Azure AD Privileged Identity Management (PIM)](https://learn.microsoft.com/en-us/entra/id-governance/privileged-identity-management/pim-configure) is a service that enables you to manage, control, and monitor access to important resources in your organization. These resources include resources in Azure Entra Id such as administrative roles and access to Azure resources.

Privileged Identity Management provides time-based and approval-based role activation to mitigate the risks of excessive, unnecessary, or misused access permissions on resources

## This notebook

This notebook will walk you through the following steps:
- Configure the group settings like notifications, group membership duration, and approval workflow
- Assign eligible groups to users

# Prerequisites

To connect to the Microsoft Graph Api you need to have an app registration in Microsoft Entra Id with the following API permissions for Microsoft Graph:
- RoleManagement.ReadWrite.Directory
- PrivilegedEligibilitySchedule.ReadWrite.AzureADGroup
- PrivilegedAccess.ReadWrite.AzureADGroup
- RoleManagementPolicy.ReadWrite.AzureADGroup

You can create one manually or run the cell below to create one.

In [ ]:
$tenantId = ""
az login -t $tenantId -o none
$appRegistration = az ad app create --display-name "Control PIM Settings" --sign-in-audience AzureADMyOrg --public-client-redirect-uris http://localhost --is-fallback-public-client true | ConvertFrom-Json
az ad app permission add --id $appRegistration.id --api 00000003-0000-0000-c000-000000000000 --api-permissions d01b97e9-cbc0-49fe-810a-750afd5527a3=Scope # RoleManagement.ReadWrite.Directory
az ad app permission add --id $appRegistration.id --api 00000003-0000-0000-c000-000000000000 --api-permissions ba974594-d163-484e-ba39-c330d5897667=Scope # PrivilegedEligibilitySchedule.ReadWrite.AzureADGroup
az ad app permission add --id $appRegistration.id --api 00000003-0000-0000-c000-000000000000 --api-permissions 32531c59-1f32-461f-b8df-6f8a3b89f73b=Scope # PrivilegedAccess.ReadWrite.AzureADGroup
az ad app permission add --id $appRegistration.id --api 00000003-0000-0000-c000-000000000000 --api-permissions 0da165c7-3f15-4236-b733-c0b0f6abe41d=Scope # RoleManagementPolicy.ReadWrite.AzureADGroup
$principal = az ad sp create --id $appRegistration.id | ConvertFrom-Json
az ad app permission admin-consent --id $appRegistration.id
"ClientId $($appRegistration.appId)"

### Set required variables

This cell sets the variables that are required to run the notebook. The variables are used in the rest of the notebook. 

- The `approversEntraIDGroup` is used in this notebook to identify the Azure Entra ID Group that contains the approvers for the PIM group membership using the Object Id.
- The `eligiblePrincipalId` is used in this notebook to identify the [Azure Entra ID group](https://learn.microsoft.com/en-us/entra/identity/role-based-access-control/groups-create-eligible?tabs=ms-powershell) or user who is able to request a group membership using the Object Id. 
- The `privilegedGroups` denotes the group names for groupmembership requests that require approval before they can become active.


In [ ]:
var tenantId = "";
var subscriptionId = "";
var approversEntraIDGroup = "";
var approversEntraIDGroupName = "";
var eligiblePrincipalId = "";

var clientId = "";
var nonPrivilegedGroups = new[] { "" };
var eligibleGroups = new[] { "" };


### Set up the Microsoft Graph SDK

The [Microsoft Graph SDK for .NET](https://learn.microsoft.com/en-us/graph/sdks/sdks-overview) allows for inspecting, creating and updating of Azure Entra ID Resources. The client is used to update PIM group membership settings and assign group memberships to groups.

In [ ]:
#r "nuget:Azure.Identity"
#r "nuget:Microsoft.Graph"

using Azure.Identity;
using Microsoft.Graph;
using Microsoft.Graph.Models;

var scopes = new[] { "RoleManagement.ReadWrite.Directory", "PrivilegedEligibilitySchedule.ReadWrite.AzureADGroup", "PrivilegedAccess.ReadWrite.AzureADGroup", "RoleManagementPolicy.ReadWrite.AzureADGroup" };
var graphOptions = new InteractiveBrowserCredentialOptions
{
	TenantId = tenantId,
	ClientId = clientId,
	AuthorityHost = AzureAuthorityHosts.AzurePublicCloud,
	RedirectUri = new Uri("http://localhost")
};

var interactiveCredential = new InteractiveBrowserCredential(graphOptions);
var graphClient = new GraphServiceClient(interactiveCredential, scopes);


### Modify default PIM group settings

This code modifies the default [PIM group settings](https://learn.microsoft.com/en-us/entra/id-governance/privileged-identity-management/groups-role-settings). The default settings are modified to allow permanent eligible assignments and sets an Azure Entra ID Group with approvers for the group membership assignments.

In [ ]:
var eligibleEntraIdGroups = (await graphClient.Groups.GetAsync((requestConfiguration) =>
{
    requestConfiguration.QueryParameters.Count = true;
	requestConfiguration.QueryParameters.Filter = string.Join(" OR ", eligibleGroups.Select(g => $"displayName eq '{g}'")); 
	requestConfiguration.QueryParameters.Select = new string []{ "id","displayName" };
    requestConfiguration.Headers.Add("ConsistencyLevel", "eventual");
})).Value.ToList();

foreach(var group in eligibleEntraIdGroups)
{
   var policies = await graphClient.Policies.RoleManagementPolicies.GetAsync((requestConfiguration) =>
    {
    	requestConfiguration.QueryParameters.Filter = $"scopeId eq '{group.Id}' and scopeType eq 'Group'";
    	requestConfiguration.QueryParameters.Expand = new string []{ "rules" };
    });

    foreach (var policy in policies.Value)
    {
        foreach (var rule in policy.Rules)
        {
            switch (rule)
            {
                case UnifiedRoleManagementPolicyApprovalRule approvalRule:
                    if (rule.Id != "Approval_EndUser_Assignment")
                        break;

                    approvalRule.Setting.IsApprovalRequired = true;

                    var stage = approvalRule.Setting.ApprovalStages.First();
                    if (!approvalRule.Setting.IsApprovalRequired.GetValueOrDefault())
                        stage.PrimaryApprovers.Clear();

                    if (stage.PrimaryApprovers.OfType<GroupMembers>().Any(gm => gm.GroupId == approversEntraIDGroup) || !approvalRule.Setting.IsApprovalRequired.GetValueOrDefault())
                        break;

                    stage.PrimaryApprovers.Add(new GroupMembers
                    {
                        Description = approversEntraIDGroupName,
                        GroupId = approversEntraIDGroup,
                    });
                    break;
                case UnifiedRoleManagementPolicyNotificationRule notificationRule:
                    switch (rule.Id)
                    {
                        case "Notification_Admin_Admin_Eligibility":
                        case "Notification_Admin_Admin_Assignment":
                        case "Notification_Admin_EndUser_Assignment":
                        case "Notification_Requestor_Admin_Eligibility":
                        case "Notification_Requestor_Admin_Assignment":
                            notificationRule.IsDefaultRecipientsEnabled = false;
                            break;
                        default:
                            notificationRule.IsDefaultRecipientsEnabled = true;
                            break;
                    }
                    break;
                case UnifiedRoleManagementPolicyExpirationRule expirationRule:
                    expirationRule.IsExpirationRequired = rule.Id != "Expiration_Admin_Eligibility";
                    break;
            }

            try
        	{
	        	var result = await graphClient.Policies.RoleManagementPolicies[policy.Id].Rules[rule.Id].PatchAsync(rule);
	        }
	        catch(Exception ex)
	        {
                policy.Display();
	    	    Console.WriteLine($"failed to update rule '{rule.Id}' for group {group.DisplayName} / policy {policy.DisplayName}: {ex.Message}");
	        }
        }
    }   
}

### Assing group membership eligibility to Azure Entra ID Group

Before a user can request a group membership, the user must be eligible for the group membership. This code assigns the Azure Entra ID Group to the group membership eligibility. The group is used to identify the users that are eligible for the group membership.

In [ ]:
var eligibleEntraIdGroups = (await graphClient.Groups.GetAsync((requestConfiguration) =>
{
    requestConfiguration.QueryParameters.Count = true;
	requestConfiguration.QueryParameters.Filter = string.Join(" OR ", eligibleGroups.Select(g => $"displayName eq '{g}'")); 
	requestConfiguration.QueryParameters.Select = new string []{ "id","displayName" };
    requestConfiguration.Headers.Add("ConsistencyLevel", "eventual");
})).Value.ToList();

foreach(var group in eligibleEntraIdGroups)
{
	var requestBody = new PrivilegedAccessGroupEligibilityScheduleRequest
	{
		Action = ScheduleRequestActions.AdminAssign,
		AccessId = PrivilegedAccessGroupRelationships.Member,
		PrincipalId = eligiblePrincipalId,
		GroupId = group.Id,
		ScheduleInfo = new RequestSchedule
		{
			Expiration = new ExpirationPattern
			{
				Type = ExpirationPatternType.NoExpiration
			}
		}
	};
	try
	{
		var result = await graphClient.IdentityGovernance.PrivilegedAccess.Group.EligibilityScheduleRequests.PostAsync(requestBody);
	}
	catch(Exception ex)
	{
		Console.WriteLine($"failed to assign group '{group.DisplayName}' ({group.Id}) to principal {eligiblePrincipalId}: {ex.Message}");
	}
};